In [1]:
import pandas as pd

df = pd.read_csv(r'data/pandas_df2_cleaned.csv', dtype='str').iloc[:, 1:]
df.head()

,0
0,len head screw x h002 custom screw len head sc...
1,cp bodi laser print wt hv fr r239 xxx use anym...
2,user se wt hv fr fr x810 print materi gener us...
3,fascia panel gv uc int white r250 inject part ...
4,cabl har operat ee sp l592 idcidc jumper cabl ...


In [2]:
df_list = df.iloc[:,0].values.tolist()
df_list[:5]

['len head screw x h002 custom screw len head screw m4x8 1300 materi extern procur h10 fcgb inserra bortolo',
 'cp bodi laser print wt hv fr r239 xxx use anymor cp bodi laser print wt45hv14fr 0318 materi potenti produc intern wd0 flcd default fdw',
 'user se wt hv fr fr x810 print materi gener user se wt45hv14fr fr 0020 materi potenti produc intern wd0 flcd default fdw',
 'fascia panel gv uc int white r250 inject part gener fascia panel gv650auc int white 300 0000 materi potenti produc intern g8a fdg neukirchnerdaniel',
 'cabl har operat ee sp l592 idcidc jumper cabl har operat ee 590 sp 26800 materi extern procur vbi fcgb schuhmach jochen']

In [3]:
from bertopic import BERTopic

topic_model = BERTopic(language="multilingual") 
topics, probs = topic_model.fit_transform(df_list)

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,161,-1_slider_anton_z20_kriznik
1,0,67,0_base_applianc_kriznik_anton
2,1,52,1_mmr_sotlar_z05_saso
3,2,45,2_cc_mum5_z41_jan
4,3,44,3_basic_applianc_y33_sbasic
5,4,33,4_fold_box_fcpr_corrug
6,5,32,5_sh_coil_coat_other
7,6,32,6_hing_top_dnak_left
8,7,31,7_hous_upper_assem_arm
9,8,31,8_box_gift_evo_bo


In [5]:
topic_model.get_document_info(df_list)[:10]

,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,len head screw x h002 custom screw len head sc...,34,34_side_cover_111500_plate,side - cover - 111500 - plate - 68200 - screw ...,0.983232,False
1,cp bodi laser print wt hv fr r239 xxx use anym...,15,15_user_manual_print_fr,user - manual - print - fr - x810 - st - st4 -...,0.181695,False
2,user se wt hv fr fr x810 print materi gener us...,15,15_user_manual_print_fr,user - manual - print - fr - x810 - st - st4 -...,0.219235,False
3,fascia panel gv uc int white r250 inject part ...,1,1_mmr_sotlar_z05_saso,mmr - sotlar - z05 - saso - gehaeusedeckel - p...,0.954154,False
4,cabl har operat ee sp l592 idcidc jumper cabl ...,10,10_wire_har_cabl_reel,wire - har - cabl - reel - cord - automat - fc...,1.000000,False
5,cabl har operat ee mm sp l592 idcidc jumper ca...,10,10_wire_har_cabl_reel,wire - har - cabl - reel - cord - automat - fc...,1.000000,False
6,wire yellow green 01 dummi wire yellowgreen 00...,10,10_wire_har_cabl_reel,wire - har - cabl - reel - cord - automat - fc...,0.413133,False
7,wire yellow green 01 dummi wire yellowgreen 00...,10,10_wire_har_cabl_reel,wire - har - cabl - reel - cord - automat - fc...,0.375291,False
8,wire yellow green 01 dummi wire yellowgreen 00...,10,10_wire_har_cabl_reel,wire - har - cabl - reel - cord - automat - fc...,0.366381,False
9,dough hook mum compl h73 spare part wire good ...,23,23_hook_dough_compl_good,hook - dough - compl - good - made - h733 - st...,0.940695,False


In [6]:
# install Jupyter Notebook Renderers if "No renderer could be found for mimetype application/vnd.plotly.v1+json"
topic_model.visualize_topics()

In [8]:
from scipy.cluster import hierarchy as sch

# Hierarchical topics
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
hierarchical_topics = topic_model.hierarchical_topics(df_list, linkage_function=linkage_function)

100%|██████████| 34/34 [00:00<00:00, 169.83it/s]


In [9]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)